Wczytanie modułów


In [288]:
import lightgbm as lgb
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import json
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import pickle


Wczytanie danych


In [289]:
data = pd.read_csv('data.csv')
#data = data_csv.sample(n=len(data_csv))
data['bmi'].fillna(data['bmi'].mean(),inplace=True)

data.head(25)

<ipython-input-289-099750243473>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['bmi'].fillna(data['bmi'].mean(),inplace=True)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.000000,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.400000,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.800000,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,28.893237,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.200000,Unknown,1


Przygotowanie danych

In [290]:
col = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]

data[col] = data[col].astype("category")


data.info()


numerical_features = ["age","hypertension","heart_disease", "avg_glucose_level", "bmi"]
categorical_features = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]
target = "stroke"

X = data[numerical_features + categorical_features]
y = data[target]




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 5110 non-null   int64   
 1   gender             5110 non-null   category
 2   age                5110 non-null   float64 
 3   hypertension       5110 non-null   int64   
 4   heart_disease      5110 non-null   int64   
 5   ever_married       5110 non-null   category
 6   work_type          5110 non-null   category
 7   Residence_type     5110 non-null   category
 8   avg_glucose_level  5110 non-null   float64 
 9   bmi                5110 non-null   float64 
 10  smoking_status     5110 non-null   category
 11  stroke             5110 non-null   int64   
dtypes: category(5), float64(3), int64(4)
memory usage: 305.3 KB


Trenowanie modelu

In [291]:
pd.options.mode.chained_assignment = None
encoder = sklearn.preprocessing.OrdinalEncoder()


X[categorical_features] = encoder.fit_transform(X[categorical_features])

print(X.head(25))

smote = SMOTE(random_state = 10)
X1, Y1 = smote.fit_resample(X, y)
Y1 = pd.DataFrame(Y1)
X1 = pd.DataFrame(X1)
print(Y1.value_counts)


X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=200)

classifier = lgb.LGBMClassifier()

# Trenowanie modelu na danych treningowych
gbm = classifier.fit(X_train, y_train)




     age  hypertension  heart_disease  avg_glucose_level        bmi  gender  \
0   67.0             0              1             228.69  36.600000     1.0   
1   61.0             0              0             202.21  28.893237     0.0   
2   80.0             0              1             105.92  32.500000     1.0   
3   49.0             0              0             171.23  34.400000     0.0   
4   79.0             1              0             174.12  24.000000     0.0   
5   81.0             0              0             186.21  29.000000     1.0   
6   74.0             1              1              70.09  27.400000     1.0   
7   69.0             0              0              94.39  22.800000     0.0   
8   59.0             0              0              76.15  28.893237     0.0   
9   78.0             0              0              58.57  24.200000     0.0   
10  81.0             1              0              80.43  29.700000     0.0   
11  61.0             0              1             12

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [292]:
y_train.value_counts()

,count
stroke,
1,3905
0,3872


In [293]:
y_pred = classifier.predict(X_test)
precision_score(y_test, y_pred)
print(X1.head())

y_probs = classifier.predict_proba(X_test)

confidence = [max(probs) for probs in y_probs]
print(confidence)


    age  hypertension  heart_disease  avg_glucose_level        bmi  gender  \
0  67.0             0              1             228.69  36.600000     1.0   
1  61.0             0              0             202.21  28.893237     0.0   
2  80.0             0              1             105.92  32.500000     1.0   
3  49.0             0              0             171.23  34.400000     0.0   
4  79.0             1              0             174.12  24.000000     0.0   

   ever_married  work_type  Residence_type  smoking_status  
0           1.0        2.0             1.0             1.0  
1           1.0        3.0             0.0             2.0  
2           1.0        2.0             0.0             2.0  
3           1.0        2.0             1.0             3.0  
4           1.0        3.0             0.0             2.0  
[0.972934049118627, 0.8318825653671186, 0.9996948844520558, 0.9964099805767285, 0.9621807313658529, 0.9646405571916533, 0.9654925973565373, 0.9840655013273351, 0.962

In [294]:
with open("model.pkl", "wb") as fout:
    pickle.dump(gbm, fout)

In [295]:
test_prob = pd.DataFrame({"age": 67, "hypertension": 0, "heart_disease": 1, "avg_glucose_level": 228.69, "gender": 1.0, "ever_married": 1.0, "work_type": 2.0, "Residence_type": 1.0, "bmi": 36.6, "smoking_status": 1.0}, index=[0])

classifier.predict(test_prob)
classifier.predict_proba(test_prob)

array([[0.23587848, 0.76412152]])

In [296]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
clf.predict(X_test)
clf.predict_proba()

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AttributeError: This 'Pipeline' has no attribute 'predict_proba'